In [1]:
# Célula 1: Importações e Configurações
import os
import logging
import pandas as pd
from query_data import extract_aneel_data
from processor_data import process_geodata, save_geodata

In [2]:
# Célula 2: Definição de Constantes e Configuração
ANEEL_URL = "https://sigel.aneel.gov.br/arcgis/rest/services/PORTAL/WFS/MapServer/0/query"
OUTPUT_FOLDER = "outputs"
FINAL_FILENAME = "aerogeradores_para_tableau.csv"

In [3]:
# Célula 3: Data Extract
raw_aneel_data = extract_aneel_data(url=ANEEL_URL)

2025-07-15 15:34:12,176 - INFO - Iniciando extração de dados da URL: https://sigel.aneel.gov.br/arcgis/rest/services/PORTAL/WFS/MapServer/0/query (com paginação)
2025-07-15 15:34:13,028 - INFO - Página obtida com 1000 registros. Total acumulado: 1000
2025-07-15 15:34:13,566 - INFO - Página obtida com 1000 registros. Total acumulado: 2000
2025-07-15 15:34:14,170 - INFO - Página obtida com 1000 registros. Total acumulado: 3000
2025-07-15 15:34:14,725 - INFO - Página obtida com 1000 registros. Total acumulado: 4000
2025-07-15 15:34:15,336 - INFO - Página obtida com 1000 registros. Total acumulado: 5000
2025-07-15 15:34:15,890 - INFO - Página obtida com 1000 registros. Total acumulado: 6000
2025-07-15 15:34:16,503 - INFO - Página obtida com 1000 registros. Total acumulado: 7000
2025-07-15 15:34:17,193 - INFO - Página obtida com 1000 registros. Total acumulado: 8000
2025-07-15 15:34:17,893 - INFO - Página obtida com 1000 registros. Total acumulado: 9000
2025-07-15 15:34:18,450 - INFO - Pági

In [4]:
# Célula 4: Processamento e Limpeza dos Dados
logging.info("\n--- INICIANDO ETAPA 2: PROCESSAMENTO DE DADOS ---")
if raw_aneel_data:
    gdf_processado = process_geodata(raw_data=raw_aneel_data)
else:
    logging.error("Processamento não pode continuar pois a extração de dados falhou.")
    gdf_processado = None

2025-07-15 15:34:26,484 - INFO - 
--- INICIANDO ETAPA 2: PROCESSAMENTO DE DADOS ---
2025-07-15 15:34:26,485 - INFO - Iniciando processamento dos dados GeoJSON.
2025-07-15 15:34:26,788 - INFO - Aplicando regra de negócio para 'Barra XI'...
2025-07-15 15:34:26,795 - INFO - Foram atualizados 12 registros de UF para 'MG' com base na regra.
2025-07-15 15:34:26,795 - INFO - Removendo quebras de linha de todas as colunas de texto...
2025-07-15 15:34:26,905 - INFO - Limpando aspas simples e duplas da coluna 'PROPRIETARIO'...
2025-07-15 15:34:26,923 - INFO - Iniciando limpeza agressiva de valores nulos e vazios...
2025-07-15 15:34:27,144 - INFO - Padronizando valores de ponto flutuante...
2025-07-15 15:34:27,149 - INFO - Valores na coluna 'DIAM_ROTOR, POT_MW, POT_MW, ALT_TOTAL' foram arredondados.
2025-07-15 15:34:27,149 - INFO - Aplicando normalização personalizada em 'POT_MW'...
2025-07-15 15:34:27,156 - INFO - Padronizando a coluna 'OPERACAO': substituindo '1' por 'Sim'...
2025-07-15 15:34:2

In [5]:
# Célula 5: Verificação dos Dados Processados
if gdf_processado is not None:
    print("\nAmostra dos dados após tratamento completo:")
    # Exibindo as colunas mais importantes para verificação
    display(gdf_processado[['NOME_EOL', 'POT_MW', 'UF', 'latitude', 'longitude']].head())
    
    print("\nDescrição estatística da 'POT_MW' após normalização precisa:")
    display(gdf_processado['POT_MW'].describe())
else:
    print("\nNão há dados para exibir.")


Amostra dos dados após tratamento completo:


,NOME_EOL,POT_MW,UF,latitude,longitude
0,Caititu,1.85,BA,-14.470968,-42.642495
1,Dreen Boa Vista,2.00,RN,-5.115754,-35.924469
2,Renascença V,2.00,RN,-5.267424,-35.844373
3,SM,2.70,RN,-5.442082,-35.923294
4,Colônia,2.10,CE,-3.540330,-38.879826



Descrição estatística da 'POT_MW' após normalização precisa:


count    23307.000000
mean         3.985530
std          1.730211
min          0.500000
25%          2.200000
50%          4.200000
75%          5.500000
max         12.000000
Name: POT_MW, dtype: float64

In [6]:
# Célula 6: Armazenamento do Arquivo Final
logging.info("\n--- INICIANDO ETAPA 3: ARMAZENAMENTO DE DADOS ---")
# Salva o GeoDataFrame final (sem a coluna de geometria) para uso no Tableau
saved_file_path = save_geodata(
    gdf=gdf_processado,
    folder=OUTPUT_FOLDER,
    filename=FINAL_FILENAME
)

if saved_file_path:
    print(f"\nPROJETO CONCLUÍDO! O arquivo final está pronto para análise em: '{saved_file_path}'")

2025-07-15 15:34:27,246 - INFO - 
--- INICIANDO ETAPA 3: ARMAZENAMENTO DE DADOS ---
2025-07-15 15:34:27,246 - INFO - Iniciando salvamento do arquivo em 'outputs/aerogeradores_para_tableau.csv'.
2025-07-15 15:34:27,513 - INFO - Arquivo salvo com sucesso em: outputs\aerogeradores_para_tableau.csv



PROJETO CONCLUÍDO! O arquivo final está pronto para análise em: 'outputs\aerogeradores_para_tableau.csv'
